In [1]:
##Hoping to remove reliance on ArcGIS for ground cover processing

# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
#import rasterio
from osgeo import gdal, gdalconst
from osgeo import ogr
import matplotlib.pyplot as plt



#Path to shapefile to use for clip/mask
pathToShapefile = 'D:/FractionalCover/MackayWhit_Buffer_10km_Albers.shp'

#Do one raster only for now
rasterPathName = 'D:/FractionalCover/GroundCover/lztmre_qld_m198909198911_dixa2.tif'

#Do one raster only for now
patchPathName = 'D:/FractionalCover/Patches/lztmre_rreef_m198909198911_dj4a2.tif'

#Final NoData Val: can't use 0, as that is a 'valid' cover val....
ourNoData = 240


In [2]:
#define a function that will interpret a raster extent matched to the shapefil bounding box
def shapeExtent_To_Raster_Box(shapeExtent, rasterGeoInfo):
    
    #Do a bunch of stuff
    originX = rasterGeoInfo[0]
    originY = rasterGeoInfo[3]
    
    pixelWidth = rasterGeoInfo[1]
    pixelHieght = rasterGeoInfo[5]
    
    clipXMin = int((shapeExtent[0] - originX) / pixelWidth)
    clipXMax = int((shapeExtent[1] - originX) / pixelWidth) + 1
    
    clipYMin = int((shapeExtent[3] - originY) / pixelHieght)
    clipYMax = int((shapeExtent[2] - originY) / pixelHieght) + 1
    
    colCount = clipXMax - clipXMin
    rowCount = clipYMax - clipYMin
    
    newLeftCoord = originX + (clipXMin * pixelWidth)
    newTopCoord = originY + (clipYMin * pixelHieght)
    
    #This is not a Geotrans object, because I also want to return the row & col count
    #and the explicit row & col numbers to be used to clip the source raster
    newGeoInfo = (newLeftCoord, newTopCoord, pixelWidth, pixelHieght, colCount, rowCount, clipXMin, clipXMax, clipYMin, clipYMax)
    
    return newGeoInfo

In [3]:
#Open shapefile
clipPolys = ogr.Open(pathToShapefile)
lyr = clipPolys.GetLayer(0)
ext = lyr.GetExtent()

#Print feature count
#lyr.GetFeatureCount()


In [4]:
#Get clip shape extent info
#MinX = ext[0], MinY = ext[3] (Southern end)
#ext

In [5]:
#Open raster
origRas = gdal.Open(rasterPathName)
origRas_GeoTrans = origRas.GetGeoTransform()

origPatchRas = gdal.Open(patchPathName)

#Print Geotrans info
origRas_GeoTrans

(404205.0, 30.0, 0.0, -1006985.0, 0.0, -30.0)

In [6]:
#Confirm no data value
#origRas.GetRasterBand(1).GetNoDataValue()


In [7]:
#Combine Geotrans info to give us  what we need
#tempOutputFile = "D:/FractionalCover/clipMask.tif"

newRasterInfo = shapeExtent_To_Raster_Box(ext, origRas_GeoTrans)
patchRasterInfo = shapeExtent_To_Raster_Box(ext, origPatchRas.GetGeoTransform())

#mem_drv = ogr.GetDriverByName('Memory')
#driver = gdal.GetDriverByName('MEM')

#temp_ds = driver.Create('', newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Byte)
##temp_ds = gdal.GetDriverByName("GTiff").Create(tempOutputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Int32)
#temp_ds.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))

#Rasterize the mask shapefile
#gdal.RasterizeLayer(temp_ds, [1], lyr, burn_values=[1])

#type(temp_ds)


In [8]:
#maskRasArray = temp_ds.ReadAsArray()
#%matplotlib inline
#plt.imshow(maskRasArray)

#print(newRasterInfo[6])
#print(newRasterInfo[7])
#print(newRasterInfo[8])
#print(newRasterInfo[9])


In [9]:
#Don't need to change the dtype from unsigned 8 bit integer just yet, and that will helkp with memory
#sourceRast = numpy.array(origRas.GetRasterBand(1).ReadAsArray(),dtype='i')
#patchRast = numpy.array(origPatchRas.GetRasterBand(1).ReadAsArray(),dtype='i')

sourceRast = origRas.GetRasterBand(1).ReadAsArray()
patchRast = origPatchRas.GetRasterBand(1).ReadAsArray()


#Important to set dtype to float, will be unsigned 8 bit inytgere, limited to vals of 255...
#clip = rast[ulY:lrY, ulX:lrX]
clipSourceRast = sourceRast[newRasterInfo[8]:newRasterInfo[9],newRasterInfo[6]:newRasterInfo[7]]
clipPatchRast = patchRast[patchRasterInfo[8]:patchRasterInfo[9],patchRasterInfo[6]:patchRasterInfo[7]]

#origNoData = origRas.GetRasterBand(1).GetNoDataValue()



#Introduce mask of NoDatas
#This will include ocean as well as gaps
#clipSourceRast = numpy.ma.masked_values(clipSourceRast, origRas.GetRasterBand(1).GetNoDataValue())
#clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
clipSourceRast = numpy.ma.masked_where((clipSourceRast < 100) | (clipSourceRast > 200) | (clipSourceRast ==  origRas.GetRasterBand(1).GetNoDataValue()), clipSourceRast)
clipPatchRast = numpy.ma.masked_where((clipPatchRast < 100) | (clipPatchRast > 200) | (clipPatchRast == origPatchRas.GetRasterBand(1).GetNoDataValue()), clipPatchRast)

#Free up some memory
sourceRast = None
patchRast = None

In [10]:
#Takes ages, doesn't change the outcome
#This will reveal that there are values outside of the range that ArcGIS tends to depict...
#print('Min: ' + str(origRas.GetRasterBand(1).GetMinimum()))
#print('Max: ' + str(origRas.GetRasterBand(1).GetMaximum()))


In [11]:
#patchRast.shape
clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
#print('Min: ' + str(clipPatchRast.min()))
#print('Max: ' + str(clipPatchRast.max()))
#print('Mean: ' + str(clipPatchRast.mean()))

In [12]:
insertedPatch = numpy.ma.masked_values(clipSourceRast.filled(clipPatchRast), origPatchRas.GetRasterBand(1).GetNoDataValue())

In [13]:
#type(clipSourceRast)
#print('Min: ' + str(insertedPatch.min()))
#print('Max: ' + str(insertedPatch.max()))
#print('Mean: ' + str(insertedPatch.mean()))

In [14]:
#Need to change data type of array so that we can go above values of 255
patchedMask = numpy.array(insertedPatch, dtype='f')
patchedMask = numpy.ma.masked_values(patchedMask, origPatchRas.GetRasterBand(1).GetNoDataValue())

In [15]:
#print('Min: ' + str(patchedMask.min()))
#print('Max: ' + str(patchedMask.max()))
#print('Mean: ' + str(patchedMask.mean()))

In [16]:
##need to adjust values, both making the values sensible, and performing the objective->visual
##Masking for 100 < value > 200 unfortunately picks up ocean areas????
#patchedMask = numpy.ma.masked_where((insertedPatch < 100) | (insertedPatch > 200), insertedPatch)

###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
#Can't use -9999!
patchedMask = patchedMask.filled(240)
patchedMask = numpy.ma.masked_values(patchedMask, 240)

In [17]:
#print('Sum: ' + str(patchedMask.sum()))
print('Min: ' + str(patchedMask.min()))
print('Max: ' + str(patchedMask.max()))
print('Mean: ' + str(patchedMask.mean()))

Min: 100.0
Max: 200.0
Mean: 112.23809801


In [18]:
#Important! Use 200.0, not '200', to ensure we retain 'float' data type
gCov = 200.0-patchedMask
#gCov = numpy.array(gCov, dtype='f')
#gCov = numpy.array(200-insertedPatch, dtype='f')
#type(gCov)
#numpy.ma.count(gCov, 0)

In [19]:
#print('Sum: ' + str(gCov.sum()))
print('Min: ' + str(gCov.min()))
print('Max: ' + str(gCov.max()))
print('Mean: ' + str(gCov.mean()))

Min: 0.0
Max: 100.0
Mean: 87.7619487312


In [20]:
visPatched = 0.00925 * numpy.power(gCov, 2)

#This essential to allow us insert NoDatas in final product... again....
visPatched = visPatched.filled(ourNoData)
visPatched = numpy.ma.masked_values(visPatched, ourNoData)

In [21]:
#print('Sum: ' + str(visPatched.sum()))
print('Min: ' + str(visPatched.min()))
print('Max: ' + str(visPatched.max()))
print('Mean: ' + str(visPatched.mean()))

Min: 0.0
Max: 92.5
Mean: 72.6535257629


In [25]:
#Cfactor....
cFactor = numpy.exp(-0.799 - (0.0474 * visPatched) + (0.000449 * numpy.power(visPatched, 2)) - (0.0000052 * numpy.power(visPatched, 3)))
cFactor = numpy.ma.masked_values(cFactor, ourNoData)

In [26]:
print('Min: ' + str(cFactor.min()))
print('Max: ' + str(cFactor.max()))
print('Mean: ' + str(cFactor.mean()))

Min: 0.00426506
Max: 0.449778
Mean: 0.0309281690302


In [28]:
#How to get this array written out as a raster??
outputFile = "D:/FractionalCover/clippedTestCov.tif"

#Integer will be fine for cover bands
#GDT_Byte is 8 bit unsigned (apparently)
#GDT_Int16
#GDT_Int32
#GDT_Float32
outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Byte)
outDS.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))


#type(outBand)

outDS.SetProjection(origRas.GetProjection())
outBand = outDS.GetRasterBand(1)

# write the data
#outBand.WriteArray(clipSourceRast, 0, 0)
outBand.WriteArray(visPatched)

# flush data to disk, set the NoData value and calculate stats
outBand.FlushCache()
outBand.SetNoDataValue(ourNoData)


outputCFactFile = "D:/FractionalCover/clippedTestCFatc.tif"
outDSCFact = gdal.GetDriverByName("GTiff").Create(outputCFactFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Float32)
outDSCFact.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))

outDSCFact.SetProjection(origRas.GetProjection())
outBandCFact = outDSCFact.GetRasterBand(1)

outBandCFact.WriteArray(cFactor)
outBandCFact.FlushCache()
outBandCFact.SetNoDataValue(ourNoData)

# georeference the image and set the projection (ROB: don't do GeoTransform, we've already done that)
##outDs.SetGeoTransform(inDs.GetGeoTransform())
outDS.SetProjection(origRas.GetProjection())
outDSCFact.SetProjection(origRas.GetProjection())

#Without this 'del', the output file doesn't get populated with values???
del outDS, outBand, outDSCFact, outBandCFact
